In [85]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime

url = "https://en.volleyballworld.com/volleyball/competitions/vnl-2023/schedule/"

res = requests.get(url)
res.encoding = "utf-8"

soup = BeautifulSoup(res.text, 'html.parser')

matches = soup.find_all("div", attrs={"class" : "vbw-mu--match"})

date = []
day = []
time_local = []
utc_time = []
time_mytime = []
status = []
gender = []
country = []
city = []
match_team = []
set_1 = []
set_2 = []
set_3 = []
set_4 = []
set_5 = []
result = []

for match in matches:
  vb_date = match.find("div", attrs={"class" : "vbw-mu__date--date"}).text
  date.append(vb_date)
  
  vb_day = match.find("div", attrs={"class" : "vbw-mu__date--day"}).text
  day.append(vb_day)

  vb_time = match.find("div", attrs={"class" : "vbw-mu__time--local"}).text
  time_local.append(vb_time)

  vb_utc = match.find("div", attrs={"class" : "vbw-mu__time--utc"}).text
  utc_time.append(vb_utc)

  vb_mytime = match.find("div", attrs={"class" : "vbw-mu__time--mytime"}).text
  time_mytime.append(vb_mytime)

  vb_status = match.find("div", attrs={"class" : "vbw-mu__status--detail"}).text 
  status.append(vb_status)

  vb_gender = match.find("div", attrs={"class" : "vbw-mu__info--details"}).text
  if vb_gender[-3] == 'M':
    gender.append(vb_gender[-3:])
  else:
    gender.append(vb_gender[-5:])
  
  vb_country = match.find("div", attrs={"class" : "vbw-mu__info--country"}).text 
  country.append(vb_country)

  vb_city = match.find("div", attrs={"class" : "vbw-mu__info--city"}).text 
  city.append(vb_city)

  team_home = match.find("div", attrs={"class" : "vbw-mu__team--home"}).find(attrs={"class" : "vbw-mu__team__name"}).text
  team_away = match.find("div", attrs={"class" : "vbw-mu__team--away"}).find(attrs={"class" : "vbw-mu__team__name"}).text
  vb_match = team_home + ' : ' + team_away
  match_team.append(vb_match)

  for i in range(1,6):
    score_home_team_set = match.find("div", attrs={"data-set-no" : f"{i}"}).find("span", attrs={"class" : "vbw-mu__pointA"}).text
    score_away_team_set = match.find("div", attrs={"data-set-no" : f"{i}"}).find("span", attrs={"class" : "vbw-mu__pointB"}).text

    score_set = score_home_team_set + ' : ' + score_away_team_set

    if i == 1:
      set_1.append(score_set)
    elif i == 2:
      set_2.append(score_set)
    elif i == 3:
      set_3.append(score_set)
    elif i == 4:
      set_4.append(score_set)
    elif i == 5:
      set_5.append(score_set)

  vb_result_home = match.find("div", attrs={"class" : "vbw-mu__score--home"}).text
  vb_result_away = match.find("div", attrs={"class" : "vbw-mu__score--away"}).text

  if vb_result_home < '0' or vb_result_away < '0':
    result.append("0 : 0")
  else:
    vb_result = vb_result_home + ' : ' + vb_result_away
    result.append(vb_result)
  

vnl_2023 = pd.DataFrame({
  "Date" : date,
  "Day" : day,
  "Local Time" : time_local,
  "UTC Time" : utc_time,
  "My Time" : time_mytime,
  "Status" : status,
  "Gender" : gender,
  "Country" : country,
  "City" : city,
  "Match" : match_team,
  "Set 1" : set_1,
  "Set 2" : set_2,
  "Set 3" : set_3,
  "Set 4" : set_4,
  "Set 5" : set_5,
  "Result" : result
})



In [86]:
vnl_2023["Date"] = pd.to_datetime(vnl_2023["Date"])
vnl_2023["Local Time"] = pd.to_datetime(vnl_2023["Local Time"], format='%H:%M')
vnl_2023["UTC Time"] = pd.to_datetime(vnl_2023["UTC Time"], format='%H:%M')
vnl_2023["My Time"] = vnl_2023['UTC Time'].dt.tz_localize('UTC').dt.tz_convert('Asia/Singapore')


In [87]:
vnl_2023["Local Time"] = vnl_2023["Local Time"].dt.time
vnl_2023["UTC Time"] = vnl_2023["UTC Time"].dt.time
vnl_2023["My Time"] = vnl_2023["My Time"].dt.time

In [88]:
vnl_2023

,Date,Day,Local Time,UTC Time,My Time,Status,Gender,Country,City,Match,Set 1,Set 2,Set 3,Set 4,Set 5,Result
0,2023-05-30,Tuesday,16:10:00,07:10:00,14:05:00,Results,Women,Japan,Nagoya,Germany : Netherlands,25 : 21,25 : 22,20 : 25,25 : 22,:,3 : 1
1,2023-05-30,Tuesday,19:40:00,10:40:00,17:35:00,Results,Women,Japan,Nagoya,Japan : Dominican Republic,25 : 23,25 : 18,22 : 25,25 : 15,:,3 : 1
2,2023-05-30,Tuesday,17:00:00,14:00:00,20:55:00,Results,Women,Türkiye,Antalya,Italy : Thailand,24 : 26,25 : 17,27 : 29,30 : 28,15 : 11,3 : 2
3,2023-05-30,Tuesday,20:15:00,17:15:00,00:10:00,Results,Women,Türkiye,Antalya,Poland : Canada,20 : 25,25 : 22,20 : 25,25 : 23,15 : 13,3 : 2
4,2023-05-31,Wednesday,15:00:00,06:00:00,12:55:00,Results,Women,Japan,Nagoya,Croatia : Bulgaria,12 : 25,17 : 25,19 : 25,:,:,0 : 3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,2023-07-20,Thursday,20:00:00,18:00:00,00:55:00,Upcoming,Men,Poland,Gdansk,Poland : TBD,:,:,:,:,:,0 : 0
204,2023-07-22,Saturday,17:00:00,15:00:00,21:55:00,Upcoming,Men,Poland,Gdansk,TBD : TBD,:,:,:,:,:,0 : 0
205,2023-07-22,Saturday,20:00:00,18:00:00,00:55:00,Upcoming,Men,Poland,Gdansk,TBD : TBD,:,:,:,:,:,0 : 0
206,2023-07-23,Sunday,17:00:00,15:00:00,21:55:00,Upcoming,Men,Poland,Gdansk,TBD : TBD,:,:,:,:,:,0 : 0
